In [1]:
import numpy as np
import tensorflow as tf

In [60]:
from keras.utils import metrics_utils
from tensorflow.python.ops import init_ops


class NegativePredictiveValue(tf.keras.metrics.Metric):

  def __init__(self, name='Negative Predictive Value', **kwargs):
    super(NegativePredictiveValue, self).__init__(name=name, **kwargs)
    self.true_negatives = self.add_weight(
        'true_negatives',
        initializer=init_ops.zeros_initializer)
    self.false_negatives = self.add_weight(
        'false_negatives',
        initializer=init_ops.zeros_initializer)


  def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.bool)
        y_pred = tf.cast(tf.round(y_pred), tf.bool)
        # if (y_pred == negative_value) and (y_pred == negative_value)
        self.true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_pred, False),
                                                  tf.equal(y_pred, y_true)), tf.float32))
        # if (y_pred == negative_value) and (y_pred != negative_value)
        self.false_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_pred, False),
                                                  tf.not_equal(y_pred, y_true)), tf.float32))

  def result(self):
      return self.true_negatives /(self.true_negatives+self.false_negatives+tf.keras.backend.epsilon())

  def reset_states(self):
        self.true_negatives.assign(0)
        self.false_negatives.assign(0)

In [61]:
m = NegativePredictiveValue()
m.update_state(np.array([0,1,0,0,0,0,0,1]),
               np.array([1,0,1,0,.1,1,1,1]))

print(float(m.result()))

tf.Tensor([False  True False False False False False  True], shape=(8,), dtype=bool)
tf.Tensor([ True False  True False False  True  True  True], shape=(8,), dtype=bool)
0.6666666865348816
